# Softmax and Cross Entropy

- Common function used in NN

## Softmax Layer

$S(y_i) = \frac{e^{y_i}}{\sum e^{y_j}}$
- Squashes the output betwee 0 and 1
- Normalized 


## Cross Entropy

Softmax function is usually combined with the cross entropy

$D(\hat{Y}, Y) = -\frac{1}{N} *\sum Y_i log(\hat{Y}_i)$
- Measures the performance of a classification model
- Can be used in multi class problems
- The better the prediciton, the lower the loss


In [1]:
import torch
import torch.nn as nn
import numpy as np


def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

x = np.array([2.0, 1.0, 0.1])
outputs = softmax(x)

print('Softmax numpy: ', outputs)

# can also be implemented in pytorch
x = torch.tensor([2.0, 1.0, 0.1])
outputs = torch.softmax(x, dim=0)

print('Softmax torch: ', outputs)


Softmax numpy:  [0.65900114 0.24243297 0.09856589]
Softmax torch:  tensor([0.6590, 0.2424, 0.0986])


In [6]:
def cross_entropy(actual, predicted):
    EPS = 1e-15
    predicted = np.clip(predicted, EPS, 1 - EPS)
    loss = -np.sum(actual * np.log(predicted))
    return loss # / float(predicted.shape[0])

# y must be one hot encoded
# if class 0: [1 0 0]
# if class 1: [0 1 0]
# if class 2: [0 0 1]
Y = np.array([1, 0, 0])
Y_pred_good = np.array([0.7, 0.2, 0.1])
Y_pred_bad = np.array([0.1, 0.3, 0.6])
l1 = cross_entropy(Y, Y_pred_good)
l2 = cross_entropy(Y, Y_pred_bad)
print(f'Loss1 numpy: {l1:.4f}')
print(f'Loss2 numpy: {l2:.4f}')


# doing the same thing in pytorch
loss = nn.CrossEntropyLoss()
# something to keep in mind with the CrossEntropyLoss
Y = torch.tensor([0]) # not one hot encoded anymore
# nsamples * nclasses = 1x3
Y_pred_good = torch.tensor([[2.0, 1.0, 0.1]])
Y_pred_bad = torch.tensor([[0.5, 2.0, 0.3]])

l1 = loss(Y_pred_good, Y)
l2 = loss(Y_pred_bad, Y)

print()
print(f'Loss1 torch: {l1:.4f}')
print(f'Loss2 torch: {l2:.4f}')


_, prediction1 = torch.max(Y_pred_good, 1)
_, prediction2 = torch.max(Y_pred_bad, 1)
print()
print(prediction1)
print(prediction2)

Loss1 numpy: 0.3567
Loss2 numpy: 2.3026

Loss1 torch: 0.4170
Loss2 torch: 1.8406

tensor([0])
tensor([1])


In [8]:
# multiple samples
Y = torch.tensor([2, 0, 1]) # not one hot encoded anymore
# nsamples * nclasses = 1x3
Y_pred_good = torch.tensor([[0.1, 1.0, 2.1],
                           [2.0, 1.0, 0.1],
                           [0.1, 3.0, 0.1]])
Y_pred_bad = torch.tensor([[2.1, 1.0, 2.1],
                           [2.0, 1.0, 2.1],
                           [0.1, 3.0, 0.1]])

l1 = loss(Y_pred_good, Y)
l2 = loss(Y_pred_bad, Y)

print()
print(f'Loss1 torch: {l1:.4f}')
print(f'Loss2 torch: {l2:.4f}')

_, prediction1 = torch.max(Y_pred_good, 1)
_, prediction2 = torch.max(Y_pred_bad, 1)
print()
print(prediction1)
print(prediction2)


Loss1 torch: 0.3018
Loss2 torch: 0.6190

tensor([2, 0, 1])
tensor([0, 2, 1])


In [9]:
# Binary classification
class NeuralNet1(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNet1, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, 1)  
    
    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        # sigmoid at the end
        y_pred = torch.sigmoid(out)
        return y_pred

model = NeuralNet1(input_size=28*28, hidden_size=5)
criterion = nn.BCELoss()

# Multiclass problem
class NeuralNet2(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet2, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        # no softmax at the end
        return out

model = NeuralNet2(input_size=28*28, hidden_size=5, num_classes=3)
criterion = nn.CrossEntropyLoss()  # (applies Softmax)